In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.spatial.transform import Rotation as R


In [2]:
def rot(k, theta):
    return R.from_rotvec(k * theta).as_matrix()

In [3]:
def rotm2eul(R):
    return R.from_matrix(R).as_euler('zyx', degrees=False)

In [4]:
def fwdkin(robot, theta, _ignore_limits = False):
    """
    Computes the pose of the robot tool flange based on a Robot object
    and the joint angles.
    
    :type    robot: Robot
    :param   robot: The robot object containing kinematic information
    :type    theta: numpy.array
    :param   theta: N x 1 array of joint angles. Must have same number of joints as Robot object
    :rtype:  Transform
    :return: The Pose of the robot tool flange    
    """    
    
    if not _ignore_limits:

        if (robot.joint_lower_limit is not None and robot.joint_upper_limit is not None):
            assert np.greater_equal(theta, robot.joint_lower_limit).all(), "Specified joints out of range"
            assert np.less_equal(theta, robot.joint_upper_limit).all(), "Specified joints out of range"
    
    p = robot.P[:,[0]]
    R = np.identity(3)
    for i in xrange(0,len(robot.joint_type)):
        if (robot.joint_type[i] == 0 or robot.joint_type[i] == 2):
            R = np.matmul(R,rot(robot.H[:,[i]],theta[i]))
        elif (robot.joint_type[i] == 1 or robot.joint_type[i] == 3):
            p = p + theta[i] * np.matmul(R,robot.H[:,[i]])
        p = p + np.matmul(R,robot.P[:,[i+1]])
        
    p=np.reshape(p,(3,))
        
    return apply_robot_aux_transforms(robot,Transform(R, p))

In [ ]:
def robotjacobiandef robotjacobian(robot, theta, _ignore_limits = False):
    """
    Computes the Jacobian matrix for the robot tool flange based on a Robot object
    and the joint angles.
    
    :type     robot: Robot
    :param    robot: The robot object containing kinematic information
    :type     theta: numpy.array
    :param    theta: N x 1 array of joint angles in radians or meters as appropriate. Must have same number of joints as Robot object.
    :rtype:   numpy.array
    :returns: The 6 x N Jacobian matrix    
    """
    
    if not _ignore_limits:
        if (robot.joint_lower_limit is not None and robot.joint_upper_limit is not None):
            assert np.greater_equal(theta, robot.joint_lower_limit).all(), "Specified joints out of range"
            assert np.less_equal(theta, robot.joint_upper_limit).all(), "Specified joints out of range"
    
    
    hi = np.zeros(robot.H.shape)
    pOi = np.zeros(robot.P.shape)
    
    p = robot.P[:,[0]]
    R = np.identity(3)
    
    pOi[:,[0]] = p
    
    H = robot.H
    P = robot.P
    joint_type = robot.joint_type
    
    for i in xrange(0, len(joint_type)):
        if (joint_type[i] == 0 or joint_type[i] == 2):
            R = np.matmul(R,rot(H[:,[i]],theta[i]))
        elif (joint_type[i] == 1 or joint_type[i] == 3):
            p = p + theta[i] * np.matmul(R,H[:,[i]])
        p = p + np.matmul(R,P[:,[i+1]])
        pOi[:,[i+1]] = p
        hi[:,[i]] = np.matmul(R,H[:,[i]])
    
    pOT = pOi[:,[len(joint_type)]]
    
    R_flange = robot.T_flange.R if (robot.T_flange is not None) else np.eye(3)

    if robot.T_flange is not None:
        pOT += np.matmul(R,np.reshape(robot.T_flange.p,(3,1)))
    if robot.p_tool is not None:
        pOT += np.matmul(np.matmul(R,R_flange),np.reshape(robot.p_tool,(3,1)))
    
    J = np.zeros([6,len(joint_type)])
    i = 0
    j = 0
    while (i < len(joint_type)):
        if (joint_type[i] == 0):
            J[0:3,[j]] = hi[:,[i]]
            J[3:6,[j]] = np.matmul(hat(hi[:,[i]]),(pOT - pOi[:,[i]]))
        elif (joint_type[i] == 1):
            J[3:6,[j]] = hi[:,[i]]
        elif (joint_type[i] == 3):
            J[3:6,[j]] = np.matmul(rot(hi[:,[i+2]], theta[i+2]),(hi[:,[i]]))
            J[0:3,[j+1]] = hi[:,[i+2]]
            J[3:6,[j+1]] = np.matmul(hat(hi[:,[i+2]]),(pOT - pOi[:,[i+2]]))
            J = J[:,0:-1]
            i = i + 2
            j = j + 1
        
        i = i + 1
        j = j + 1

    if not robot.T_base:
        return J
    else:
        R_J = np.block([[robot.T_base.R, np.zeros((3,3))],[np.zeros((3,3)), robot.T_base.R]])
        return np.matmul(R_J,J)

In [ ]:
def qprimelimits_full(qlimit, qprev, N, qpmax, qpmin):
    n = len(qlimit)
    lb_js = N * (qlimit[:,0] - qprev)
    ub_js = N * (qlimit[:,1] - qprev)
    lb = np.maximum(lb_js, qpmin)
    ub = np.minimum(ub_js, qpmax)
    return lb, ub

In [ ]:
def getqp_H(dq, J, vr, vp, er, ep):
    n = len(dq)
    H1 = np.dot(np.hstack([J, np.zeros((6,2))]).T, np.hstack([J, np.zeros((6,2))]))
    H2 = np.dot(np.hstack([np.zeros((3,n)), vr.reshape(-1,1), np.zeros((3,1))]).T,
                np.hstack([np.zeros((3,n)), vr.reshape(-1,1), np.zeros((3,1))]))
    H3 = -2 * np.dot(np.hstack([J, np.zeros((6,2))]).T,
                     np.hstack([np.zeros((3,n)), vr.reshape(-1,1), np.zeros((3,1))]))
    H3 = (H3 + H3.T) / 2
    H4 = np.dot(np.array([[np.sqrt(er)], [np.sqrt(ep)]]).T,
                np.array([[np.sqrt(er)], [np.sqrt(ep)]]))
    H = 2 * (H1 + H2 + H3 + H4)
    return H

In [ ]:
def getqp_f(dq, er, ep):
    return -2 * np.array([np.zeros(len(dq)), er, ep]).T

In [ ]:
def quadprog(H, f, A, b, Aeq, beq, lb, ub, x0, options):
    # Define the objective function
    fun = lambda x: 0.5 * np.dot(x.T, np.dot(H, x)) + np.dot(f.T, x)
    # Define the constraints
    cons = ({'type': 'eq', 'fun': lambda x: np.dot(Aeq, x) - beq},
            {'type': 'ineq', 'fun': lambda x: b - np.dot(A, x)})
    # Define the bounds
    bounds = [(l, u) for l, u in zip(lb, ub)]
    # Solve the quadratic program
    res = minimize(fun, x0, method='SLSQP', bounds=bounds, constraints=cons, options=options)
    return res.x, res.success

In [ ]:
def qpPathGen(robot, q0, P0Td, R0Td, epsilon_r, epsilon_p, q_prime_min, q_prime_max, N):
    n = len(q0)
    lambda_ = np.linspace(0, 1, N + 1)
    options = {'disp': False}
    R0T0, P0T0 = fwdkin(robot, q0)
    ER0 = np.dot(R0T0, R0Td.T)
    k_hat, theta0 = R.from_matrix(ER0).as_rotvec()
    Euldes_lambda = np.zeros((3, len(lambda_)))
    Pdes_lambda = np.zeros((3, len(lambda_)))
    dP0T_dlambda = P0Td - P0T0
    der_dlambda = np.zeros(len(lambda_))

In [ ]:
robot = YourRobotClass() # Define your robot
q0 = np.array([initial joint states])
P0Td = np.array([target position])
R0Td = np.array([target orientation])
epsilon_r, epsilon_p, q_prime_min, q_prime_max, N = [your parameters]
q_lambda, lambda_, P0T_lambda, R0T_lambda = qpPathGen(robot, q0, P0Td, R0Td